# Processo de geração e avaliação de multiplas respostas sobre design de visualização de dados com LLMs

O processo de geração é algo que pode ser divido em algumas partes que ajudam a construir o produto final a ser coletado.

Para este trabalho, o objetivo é utilizar o grande modelo de linguagem da empresa OpenAI para gerar perguntas e respostas que auxiliem no processo de design de visualização de dados.

### Cenário

Imagine o seguinte cenário:

<i>Você é gerente de uma empresa de venda de eletronicos e que tem lojas espalhadas por todo o Brasil. O seu gerente do setor de smartphones, acompanhando os lançamentos dos dispositivos, avisa que será lançado um novo dispositivo Apple. Para distribuir os novos dispositivos, o seu gerente deverá apresentar quais lojas vendem mais, assim você poderá mandar mais dispositivos para o estoque.<i>



Desejos na visualização:
- Usuário: Gerente de tecnologia.
- Desejo: Mostrar as lojas que mais vendem smartphones Apple durante o ultimo ano

Entendendo esses pontos, é possível que sejam construídos prompts para o modelo de linguagem. O intuíto é que entendo o contexto do usuário e os desejos dele o sistema possa auxiliar no design da visualização.

Para isso podemos usar o seguinte modelo:

<b>"Eu, como um [usuário], preciso [desejo], qual seria o melhor tipo de visualização para fazer isso?"</b>

<i>"Eu, como um gerente de tecnologia, preciso mostrar as lojas que mais vendem smartphones Apple durante o ultimo ano, qual seria o melhor tipo de visualização para fazer isso?"</i>

### Montando as perguntas

Vamos construir um prompt para criar os usuários, os desejos para então realizar as perguntas para o modelo. Com esse prompt podemos coleta uma lista com as sugestões de visualização de dados que o modelo acredita que poderia suprir a necessidade do usuário.

- Denfinindo a função do assistente para pergunta

In [1]:
pergunta = [
    {
        # aqui definimos que a resposta deve vir no formato JSON
        "role": "system",
        "content": "Você, como um assistente para gerar cenários, deverá construir personagens e desejos para uma visualização de dados e deverá responder a pergunta com uma lista de possibilidades de visualização de dados como resposta e retornar em uma lista no formato JSON"
    },
    {
        # aqui definimos o como deverá ser a pergunta
        "role": "user",
        "content": "A pergunta de visualização de dados deverá seguir o seguinte padrão: Eu, como um [usuário], preciso [desejo], qual seria o melhor tipo de visualização para fazer isso?"
    },
    
    {
        # aqui definimos o padrão de resposta em cada item da lista
        "role": "user",
        "content": " Para cada pergunta deverá ser retornada uma lista com sugestões de resposta, cada item da lista de sugestões deve vir no formato: {/tipo: [tipo da visualuação de dados], descrição: [descrição clara do motivo de usar com pelo menos 50 palavras], variaveis: [lista com as variáveis necessárias para usar essa visualização e como usar a variável] }/"
    },
    
    {
        # aqui definimos o como deverá ser a pergunta
        "role": "assistant",
        "content": "Para cada pergunta o sistema deve responder no seguinte formato: {/ pergunta: [pergunta formulada seguindo o padrão ], respostas: [lista de sugestões de visualização de dados para a pergunta seguindo o padrão com pelo menos 3 opções] }/"
    },
    
    {
        # aqui definimos o como deverá ser a pergunta
        "role": "assistant",
        "content": "Gere para mim 20 perguntas e listas de respostas seguindo os padrões definidos em uma lista JSON"
    },
]
print(pergunta, )

[{'role': 'system', 'content': 'Você, como um assistente para gerar cenários, deverá construir personagens e desejos para uma visualização de dados e deverá responder a pergunta com uma lista de possibilidades de visualização de dados como resposta e retornar em uma lista no formato JSON'}, {'role': 'user', 'content': 'A pergunta de visualização de dados deverá seguir o seguinte padrão: Eu, como um [usuário], preciso [desejo], qual seria o melhor tipo de visualização para fazer isso?'}, {'role': 'user', 'content': ' Para cada pergunta deverá ser retornada uma lista com sugestões de resposta, cada item da lista de sugestões deve vir no formato: {/tipo: [tipo da visualuação de dados], descrição: [descrição clara do motivo de usar com pelo menos 50 palavras], variaveis: [lista com as variáveis necessárias para usar essa visualização e como usar a variável] }/'}, {'role': 'assistant', 'content': 'Para cada pergunta o sistema deve responder no seguinte formato: {/ pergunta: [pergunta formul

- Importando o ChatGPT

In [2]:
from openai import OpenAI # type: ignore
client = OpenAI()

- Criando uma função para executar o Chat

In [3]:
def executarChat(mensagem):
  return client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={ "type": "json_object" },
    messages=mensagem,
  )

- Executando o chat com o modelo e o formato para as perguntas e respostas

In [4]:
responsePerguntas = executarChat(pergunta).choices[0].message.content

print(responsePerguntas)

{
  "perguntas": [
    {
      "pergunta": "Eu, como um analista de marketing, preciso visualizar a distribuição de idade dos clientes, qual seria o melhor tipo de visualização para fazer isso?",
      "respostas": [
        {
          "tipo": "Histograma",
          "descricao": "Um histograma é uma visualização gráfica de dados que usa barras para representar a frequência ou a distribuição de um conjunto de dados. Ao plotar a distribuição de idade dos clientes em um histograma, é possível identificar os intervalos de idade mais comuns e observar a distribuição geral da idade dos clientes.",
          "variaveis": ["Idade dos clientes"],
          "como_usar": "Para utilizar um histograma, basta plotar os dados de idade dos clientes no eixo x e a frequência no eixo y."
        },
        {
          "tipo": "Gráfico de barras",
          "descricao": "Um gráfico de barras é uma excelente forma de visualizar a distribuição de idade dos clientes, pois permite comparar facilmente difere

- Guardando as informações da pergunta em um JSON

In [20]:
import json
from datetime import datetime

hora = datetime.today().strftime('%d_%m_%y_%H_%M_%S')
nomeArquivo = "./perguntas/responsePerguntas_"+hora+".json"

if responsePerguntas is not None:
  jsonConverted = json.loads(responsePerguntas)
  with open(nomeArquivo, "w", encoding='utf-8') as outfile:
    json.dump(jsonConverted, outfile, ensure_ascii=False)
  print("Dados salvos em" + nomeArquivo)

Dados salvos em./perguntas/responsePerguntas_22_04_24_11_21_50.json


- Definindo a função do assistente para avaliar as respostas

In [35]:
resposta = [
    {
        # aqui definimos que a resposta deve vir no formato JSON
        "role": "system",
        "content": "Você como um assistente no processo de design de visualização de dados, deverá avaliar as possibilidades de resposta e responder com uma classificação de qual seria a mais indicada em formato JSON"
    },
    
    {
        "role": "user",
        "content": "Você receberá um JSON com o seguinte formato: perguntas : {/ pergunta : [pergunta de design de visualização de dados], respostas: [lista de respostas para a pergunta com o seguinte formato {/ tipo: [tipo de visualização de dados], descrição: [descrição da visualização para a pergunta], variaveis: [lista de variáveis necessárias para construir a visualização] }/ ] }/"
    },
    
    {
        "role": "user",
        "content": "Você deverá avaliar cada uma das respostas de visualização de dados e retornar uma lista com as seguintes informações: {/ pergunta: [pergunta de design de visualização de dados], avaliações: [lista de avaliações das respostas para a visualização]}/"
    },
    
    {
        "role": "user",
        "content": "As avaliações das resposta para cada pergunta deverá ter o seguinte formato {/ tipo: [tipo da visualuação de dados], descrição: [descrição da visualização para a pergunta], variaveis: [lista de variáveis necessárias para construir a visualização], nota: [uma nota de 1-7 para a visualização], motivo: [descrever o motivo da nota em relação as demais opções] }/ }/"
    },
    
    {
        "role": "user",
        "content": "avalie cada uma das respostas de visualização de dados e retorne uma lista com as perguntas e respostas no processo de design de visualização de dados a seguir: " + responsePerguntas # type: ignore
     }
]

print(resposta)

[{'role': 'system', 'content': 'Você como um assistente no processo de design de visualização de dados, deverá avaliar as possibilidades de resposta e responder com uma classificação de qual seria a mais indicada em formato JSON'}, {'role': 'user', 'content': 'Você receberá um JSON com o seguinte formato: perguntas : {/ pergunta : [pergunta de design de visualização de dados], respostas: [lista de respostas para a pergunta com o seguinte formato {/ tipo: [tipo de visualização de dados], descrição: [descrição da visualização para a pergunta], variaveis: [lista de variáveis necessárias para construir a visualização] }/ ] }/'}, {'role': 'user', 'content': 'Você deverá avaliar cada uma das respostas de visualização de dados e retornar uma lista com as seguintes informações: {/ pergunta: [pergunta de design de visualização de dados], avaliações: [lista de avaliações das respostas para a visualização]}/'}, {'role': 'user', 'content': 'As avaliações das resposta para cada pergunta deverá ter 

- Executando o chat com o modelo e o formato para avaliar as respostas

In [36]:
responseRespostas = executarChat(resposta).choices[0].message.content

print(responseRespostas)

{
  "perguntas": [
    {
      "pergunta": "Eu, como um analista de marketing, preciso visualizar a distribuição de idades dos nossos clientes, qual seria o melhor tipo de visualização para fazer isso?",
      "avaliações": [
        {
          "tipo": "Gráfico de Barras",
          "descrição": "O gráfico de barras é uma excelente opção para visualizar a distribuição de idades, pois permite comparar facilmente diferentes faixas etárias.",
          "variaveis": ["Faixa Etária", "Número de Clientes"],
          "nota": 5,
          "motivo": "Permite comparação de diferentes faixas etárias, mas pode não ser tão eficiente para distribuições detalhadas."
        },
        {
          "tipo": "Histograma",
          "descrição": "O histograma é uma visualização ideal para entender a distribuição de idades de forma mais detalhada.",
          "variaveis": ["Idade do Cliente", "Contagem de Clientes"],
          "nota": 7,
          "motivo": "É a melhor opção para visualizar a distribuiçã

- Guardando as informações de avaliação de resposta em um JSON

In [46]:
if responseRespostas is not None:
  jsonConverted = json.loads(responseRespostas)
  with open("responseRespostas.json", "w", encoding='utf-8') as outfile:
    json.dump(jsonConverted, outfile, ensure_ascii=False)
  print("Dados salvos em responseRespostas.json")

Dados salvos em responseRespostas.json


- Convertendo respostas para CSV

In [60]:
import csv

with open("responseRespostas.json", "r", encoding='utf-8') as inputfile:
    datas = json.load(inputfile)

for data in datas['perguntas']:
    for evaluation in data['avaliações']:
        row = [data['pergunta'],evaluation['tipo'], evaluation['descrição'], evaluation['variaveis'], evaluation['nota'], evaluation['motivo']]

# # Abrir um arquivo CSV para escrita
with open('responseRespostas.csv', 'w', newline='', encoding='utf-8') as file:
    # Criar um objeto escritor CSV
    csv_writer = csv.writer(file)
    print('Arquivo criado')
    
    # Escrever a linha de cabeçalho
    header = ['Pergunta', 'Tipo', 'Descrição', 'Variáveis', 'Nota', 'Motivo']
    csv_writer.writerow(header)
    print('Cabeçalho criado')
    
#     # Escrever as linhas de dados
    for data in datas['perguntas']:
        for evaluation in data['avaliações']:
            row = [data['pergunta'],evaluation['tipo'], evaluation['descrição'], evaluation['variaveis'], evaluation['nota'], evaluation['motivo']]
            csv_writer.writerow(row)
    print('Informações preenchidas')

Arquivo criado
Cabeçalho criado
Informações preenchidas
